In [5]:
import torch
import os
import random
import argparse
from dataclasses import dataclass, field
import transformers
from torch.utils.data import Dataset, DataLoader
import pdb
import pandas as pd
from transformers import AutoImageProcessor, AutoModelForImageClassification
from tqdm import tqdm
import random
import numpy as np
import torch
import cv2
import torchvision.transforms as T
from PIL import Image
from sklearn.metrics import classification_report, accuracy_score

import torch.nn as nn

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)
cache_dir = '/home/mila/s/soroush.omranpour/scratch/hf_cache'

In [2]:
class FakeClueDataset(Dataset):
    def __init__(self, data_dir, processor, train=False):
        super().__init__()
        self.data_dir = os.path.join(data_dir, 'train' if train else 'test')
        json_path = os.path.join(data_dir, f'data_json/{"train" if train else "test"}.json')
        self.df = pd.read_json(json_path)
        self.df = self.df[(self.df.cate != 'doc') & (self.df.cate != 'satellite')]
        self.df.reset_index(inplace=True, drop=True)
        self.processor = processor

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        img_path = os.path.join(self.data_dir, self.df.loc[idx, 'image'])
        label = 1 - self.df.loc[idx, 'label'] ## in the dataset real=1 and fake=0
        
        image = Image.open(img_path)
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)
        image = image.convert("RGB")
        inputs = self.processor(image, return_tensors="pt")['pixel_values'][0]
        return inputs, label

In [9]:
import safetensors
from transformers import AutoModelForImageClassification, AutoConfig
from safetensors.torch import load_file

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# processor = AutoImageProcessor.from_pretrained(
#     "microsoft/swinv2-small-patch4-window16-256", cache_dir=cache_dir, use_fast=True
# )
processor = AutoImageProcessor.from_pretrained(
    "facebook/convnextv2-base-22k-224", cache_dir=cache_dir, use_fast=True
)
model = AutoModelForImageClassification.from_pretrained('weights/convnext').eval().cuda()

preprocessor_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

In [6]:
dataset = FakeClueDataset(
    data_dir="/home/mila/s/soroush.omranpour/scratch/FakeClue", 
    processor=processor, 
    train=False
)
print(len(dataset))
test_dataloader = DataLoader(
    dataset,
    batch_size=66,
    shuffle=False,
    num_workers=6,
    pin_memory=True,
)

3549


In [ ]:
from tqdm.auto import tqdm

gt = []
pred = []
with torch.inference_mode():
    for x, y in tqdm(test_dataloader):    
        pred += [model(x.to(device)).logits.argmax(-1).detach().cpu()]
        gt += [y]
    gt = torch.cat(gt)
    pred = torch.cat(pred)  

  0%|          | 0/54 [00:00<?, ?it/s]

In [8]:
print(classification_report(gt, pred, digits=4))

              precision    recall  f1-score   support

           0     0.5503    0.9937    0.7083      1260
           1     0.9937    0.5531    0.7106      2289

    accuracy                         0.7095      3549
   macro avg     0.7720    0.7734    0.7095      3549
weighted avg     0.8363    0.7095    0.7098      3549



In [18]:
gt.sum(), pred.sum()

(tensor(3192), tensor(1539))